In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
df = pd.read_csv("./food-allergy-analysis-Zenodo.csv")
df.head()

,SUBJECT_ID,BIRTH_YEAR,GENDER_FACTOR,RACE_FACTOR,ETHNICITY_FACTOR,PAYER_FACTOR,ATOPIC_MARCH_COHORT,AGE_START_YEARS,AGE_END_YEARS,SHELLFISH_ALG_START,...,CASHEW_ALG_END,ATOPIC_DERM_START,ATOPIC_DERM_END,ALLERGIC_RHINITIS_START,ALLERGIC_RHINITIS_END,ASTHMA_START,ASTHMA_END,FIRST_ASTHMARX,LAST_ASTHMARX,NUM_ASTHMARX
0,1,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P1 - Medicaid,False,0.093087,3.164956,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1994,S1 - Female,R0 - White,E0 - Non-Hispanic,P0 - Non-Medicaid,False,12.232717,18.880219,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.262834,18.880219,2.0
2,3,2006,S0 - Male,R0 - White,E1 - Hispanic,P0 - Non-Medicaid,True,0.010951,6.726899,NaN,...,NaN,4.884326,NaN,3.917864,6.157426,5.127995,NaN,1.404517,6.157426,4.0
3,4,2004,S0 - Male,R4 - Unknown,E1 - Hispanic,P0 - Non-Medicaid,False,2.398357,9.111567,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2006,S1 - Female,R1 - Black,E0 - Non-Hispanic,P0 - Non-Medicaid,False,0.013689,6.193018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_accuracy(y_pred,y_test):
    correctly_classified=0
    for count in range(np.size(y_pred)) :
        if y_test[count]==y_pred[count]:
           correctly_classified=correctly_classified+1
    acc=(correctly_classified/len(y_pred))*100
    print("Accuracy on test set is {:.2f}".format(acc))
    return acc

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333200 entries, 0 to 333199
Data columns (total 50 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   SUBJECT_ID               333200 non-null  int64  
 1   BIRTH_YEAR               333200 non-null  int64  
 2   GENDER_FACTOR            333200 non-null  object 
 3   RACE_FACTOR              333200 non-null  object 
 4   ETHNICITY_FACTOR         333200 non-null  object 
 5   PAYER_FACTOR             333200 non-null  object 
 6   ATOPIC_MARCH_COHORT      333200 non-null  bool   
 7   AGE_START_YEARS          333200 non-null  float64
 8   AGE_END_YEARS            333200 non-null  float64
 9   SHELLFISH_ALG_START      5246 non-null    float64
 10  SHELLFISH_ALG_END        1051 non-null    float64
 11  FISH_ALG_START           1796 non-null    float64
 12  FISH_ALG_END             527 non-null     float64
 13  MILK_ALG_START           7289 non-null    float64
 14  MILK

Preparing the dataset

In [8]:
#We use the get_dummies function 
df=pd.get_dummies(df,columns=["GENDER_FACTOR","ETHNICITY_FACTOR","RACE_FACTOR","PAYER_FACTOR","ATOPIC_MARCH_COHORT"],dtype=int)
df.drop(["GENDER_FACTOR_S1 - Female","ETHNICITY_FACTOR_E0 - Non-Hispanic","PAYER_FACTOR_P0 - Non-Medicaid","ATOPIC_MARCH_COHORT_False"],axis="columns",inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333200 entries, 0 to 333199
Data columns (total 54 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   SUBJECT_ID                                  333200 non-null  int64  
 1   BIRTH_YEAR                                  333200 non-null  int64  
 2   AGE_START_YEARS                             333200 non-null  float64
 3   AGE_END_YEARS                               333200 non-null  float64
 4   SHELLFISH_ALG_START                         5246 non-null    float64
 5   SHELLFISH_ALG_END                           1051 non-null    float64
 6   FISH_ALG_START                              1796 non-null    float64
 7   FISH_ALG_END                                527 non-null     float64
 8   MILK_ALG_START                              7289 non-null    float64
 9   MILK_ALG_END                                4580 non-null    float64
 

In [10]:
df.head()

,SUBJECT_ID,BIRTH_YEAR,AGE_START_YEARS,AGE_END_YEARS,SHELLFISH_ALG_START,SHELLFISH_ALG_END,FISH_ALG_START,FISH_ALG_END,MILK_ALG_START,MILK_ALG_END,...,NUM_ASTHMARX,GENDER_FACTOR_S0 - Male,ETHNICITY_FACTOR_E1 - Hispanic,RACE_FACTOR_R0 - White,RACE_FACTOR_R1 - Black,RACE_FACTOR_R2 - Asian or Pacific Islander,RACE_FACTOR_R3 - Other,RACE_FACTOR_R4 - Unknown,PAYER_FACTOR_P1 - Medicaid,ATOPIC_MARCH_COHORT_True
0,1,2006,0.093087,3.164956,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,1,0,0,0,1,0
1,2,1994,12.232717,18.880219,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0,0,1,0,0,0,0,0,0
2,3,2006,0.010951,6.726899,NaN,NaN,NaN,NaN,1.002053,1.002053,...,4.0,1,1,1,0,0,0,0,0,1
3,4,2004,2.398357,9.111567,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,0,0,0,0,1,0,0
4,5,2006,0.013689,6.193018,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,1,0,0,0,0,0


In [28]:

df=df.fillna(df.mean())
print(df.isna().sum().sum())

0


Splitting the data


In [29]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1:].values

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [30]:

sk_model = LogisticRegression()

In [31]:

sk_model.fit(x_train,y_train)

c:\Users\asus zephyrus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [32]:

y_pred=sk_model.predict(x_test)

In [33]:

acc=get_accuracy(y_pred,y_test)

Accuracy on test set is 90.93
